In [2]:
%matplotlib inline

import pandas
import matplotlib.pyplot as plt 
from sklearn import pipeline, preprocessing, cross_validation

In [3]:
features = pandas.read_csv( 'paper_draft/svm/feature_sets/total_score.csv', index_col=0 )
features.head()

,total_score,fa_rep,hbond_sc,tot_pstat_pm,tot_nlpstat_pm,tot_burunsat_pm,tot_hbond_pm,tot_NLconts_pm,tot_nlsurfaceE_pm,SR_1_total_score,...,SR_4_burunsat_pm,SR_4_pstat_pm,SR_4_nlpstat_pm,SR_5_total_score,SR_5_fa_rep,SR_5_hbond_sc,SR_5_interf_E_1_2,SR_5_dsasa_1_2,SR_5_hbond_pm,SR_5_burunsat_pm
name,,,,,,,,,,,,,,,,,,,,,
A192S,-711.382,278.595,-73.288,0.702,0.694,143.5,440.3,148.8,0.723,1.872,...,0.0,0.840,0.887,-5.751,0.882,-3.765,-11.503,0.901,7.8,1.0
A227W,6520.576,7494.507,-71.944,0.698,0.696,146.2,439.0,149.6,0.750,1.884,...,0.0,0.817,0.768,-5.740,0.886,-3.725,-11.479,0.902,7.7,1.0
A236E,737.532,1683.342,-72.075,0.691,0.691,147.0,438.8,149.4,0.679,1.903,...,0.0,0.860,0.854,-5.743,0.891,-3.760,-11.488,0.901,7.8,1.0
A249E,-707.884,282.056,-72.303,0.705,0.697,145.3,439.3,150.0,0.706,1.900,...,0.0,0.793,0.849,-5.616,0.903,-3.674,-11.236,0.902,7.7,1.0
A356A,-711.648,278.723,-72.366,0.696,0.695,144.2,439.4,148.2,0.724,1.896,...,0.0,0.800,0.835,-5.877,0.866,-3.877,-11.754,0.902,7.9,1.0


In [4]:
express = pandas.read_csv( 'paper_draft/data_set.csv', index_col='mutant', usecols=['mutant','expression'] )

J = express.join( features ).dropna()
X = J.drop( 'expression', axis=1 )
y = J.expression

print X.shape
print y.shape

(107, 44)
(107,)


In [5]:
def my_function( data_set ):
    '''X, y = my_function( data_set ) '''
    
    features = pandas.read_csv( data_set, index_col=0 )
    express = pandas.read_csv( 'paper_draft/data_set.csv', index_col='mutant', usecols=['mutant','expression'] )
    J = express.join( features ).dropna()
    X = J.drop( 'expression', axis=1 )
    y = J.expression

    return X, y 

my_X, my_y = my_function( 'paper_draft/svm/feature_sets/total_score.csv' ) 

print my_X.shape
print my_y.shape 

(107, 44)
(107,)


In [12]:
from glob import glob 
from sklearn import svm, decomposition 
from numpy import unique 

for data_set in glob( 'paper_draft/svm/feature_sets/*.csv' ):
    X_train, y_train = my_function( data_set ) 
    pln = pipeline.Pipeline([ ( 'PCA', decomposition.PCA() ), ( 'SVM', svm.SVC( class_weight='balanced' ) ), ]) 
    scores = cross_validation.cross_val_score( pln, X_train, y_train )
    preds = cross_validation.cross_val_predict( pln, X_train, y_train ) 
    print data_set, scores.mean().round( 3 ), '±',  scores.std().round( 3 ) 
    print unique( preds, return_counts=True ) 

paper_draft/svm/feature_sets/fa_rep.csv 0.572 ± 0.142
(array([0, 1]), array([34, 73]))
paper_draft/svm/feature_sets/hbond_sc.csv 0.572 ± 0.121
(array([0, 1]), array([40, 67]))
paper_draft/svm/feature_sets/SR_1_burunsat_pm.csv 0.599 ± 0.102
(array([0, 1]), array([29, 78]))
paper_draft/svm/feature_sets/SR_1_fa_rep.csv 0.618 ± 0.089
(array([0, 1]), array([31, 76]))
paper_draft/svm/feature_sets/SR_1_hbond_pm.csv 0.59 ± 0.108
(array([0, 1]), array([30, 77]))
paper_draft/svm/feature_sets/SR_1_hbond_sc.csv 0.59 ± 0.111
(array([0, 1]), array([30, 77]))
paper_draft/svm/feature_sets/SR_1_nlpstat_pm.csv 0.636 ± 0.059
(array([0, 1]), array([31, 76]))
paper_draft/svm/feature_sets/SR_1_pstat_pm.csv 0.599 ± 0.102
(array([0, 1]), array([35, 72]))
paper_draft/svm/feature_sets/SR_1_total_score.csv 0.609 ± 0.116
(array([0, 1]), array([36, 71]))
paper_draft/svm/feature_sets/SR_2_burunsat_pm.csv 0.571 ± 0.124
(array([0, 1]), array([32, 75]))
paper_draft/svm/feature_sets/SR_2_fa_rep.csv 0.569 ± 0.044
(array

In [22]:
from sklearn import grid_search

my_data_set = 'paper_draft/svm/feature_sets/SR_2_pstat_pm.csv' 

my_list = [ 0.0003, 0.003, 0.03, 0.3, 3., 30., 300., 3000., ] 
param_grid = { 
    'pca__n_components': [ 2, 4, 8, 16 ],
    'SVM__gamma': my_list, 
    'SVM__C': my_list, 
}

for data_set in [ my_data_set ]:
    X_train, y_train = my_function( data_set ) 
    pln = pipeline.Pipeline([ ( 'scaler', preprocessing.StandardScaler() ), ( 'PCA', decomposition.PCA() ), ( 'SVM', svm.SVC() ), ]) 
    scores = cross_validation.cross_val_score( pln, X_train, y_train )
    preds = cross_validation.cross_val_predict( pln, X_train, y_train ) 
    print data_set, scores.mean().round( 3 ), '±',  scores.std().round( 3 ) 
    print unique( preds, return_counts=True ) 
    
    gs = grid_search.GridSearchCV( pln, param_grid, cv=10 ) # SKF by default 
    gs.fit( X_train, y_train ) 
    score = gs.score( X_train, y_train )
    preds = gs.predict( X_train, y_train ) 

paper_draft/svm/feature_sets/SR_2_pstat_pm.csv 0.738 ± 0.006
(array([1]), array([107]))


AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [ ]:
X_train, y_train = my_function